In [27]:
from tkinter import *
import random
import math

In [28]:
WIDTH = 800
HEIGHT = 400
PAD_W = 15
PAD_H = 80
BALL_SIZE = 20
BALL_SPEED = 5
PAD_SPEED = 6
WINNING_SCORE = 5
FPS = 60

In [29]:
ball_x = WIDTH // 2
ball_y = HEIGHT // 2
ball_dx = BALL_SPEED * random.choice([-1, 1])
ball_dy = BALL_SPEED * random.choice([-1, 1])

left_pad_y = HEIGHT // 2 - PAD_H // 2
right_pad_y = HEIGHT // 2 - PAD_H // 2

player1_score = 0
player2_score = 0

left_up = False
left_down = False
right_up = False
right_down = False

game_paused = False
game_over = False

In [30]:
def reset_ball():
    global ball_x, ball_y, ball_dx, ball_dy
    ball_x = WIDTH // 2
    ball_y = HEIGHT // 2
    speed = BALL_SPEED + (player1_score + player2_score) * 0.5
    angle = random.uniform(-math.pi/4, math.pi/4)
    direction = random.choice([-1, 1])
    ball_dx = speed * direction * math.cos(angle)
    ball_dy = speed * math.sin(angle)

In [31]:
def update_ball():
    global ball_x, ball_y, ball_dx, ball_dy, player1_score, player2_score, game_over
    
    ball_x += ball_dx
    ball_y += ball_dy
    
    if ball_y <= BALL_SIZE//2 or ball_y >= HEIGHT - BALL_SIZE//2:
        ball_dy = -ball_dy
        ball_y = max(BALL_SIZE//2, min(HEIGHT - BALL_SIZE//2, ball_y))
    
    if ball_x <= PAD_W + BALL_SIZE//2:
        if left_pad_y <= ball_y <= left_pad_y + PAD_H:
            ball_dx = abs(ball_dx)
            relative_intersect_y = (ball_y - (left_pad_y + PAD_H//2)) / (PAD_H//2)
            ball_dy = ball_dx * relative_intersect_y * 0.75
        else:
            player2_score += 1
            if player2_score >= WINNING_SCORE:
                game_over = True
            else:
                reset_ball()
    
    elif ball_x >= WIDTH - PAD_W - BALL_SIZE//2:
        if right_pad_y <= ball_y <= right_pad_y + PAD_H:
            ball_dx = -abs(ball_dx)
            relative_intersect_y = (ball_y - (right_pad_y + PAD_H//2)) / (PAD_H//2)
            ball_dy = abs(ball_dx) * relative_intersect_y * 0.75
        else:
            player1_score += 1
            if player1_score >= WINNING_SCORE:
                game_over = True
            else:
                reset_ball()

In [32]:
def update_paddles():
    global left_pad_y, right_pad_y
    
    if left_up and left_pad_y > 0:
        left_pad_y -= PAD_SPEED
    if left_down and left_pad_y < HEIGHT - PAD_H:
        left_pad_y += PAD_SPEED
    if right_up and right_pad_y > 0:
        right_pad_y -= PAD_SPEED
    if right_down and right_pad_y < HEIGHT - PAD_H:
        right_pad_y += PAD_SPEED
    
    left_pad_y = max(0, min(HEIGHT - PAD_H, left_pad_y))
    right_pad_y = max(0, min(HEIGHT - PAD_H, right_pad_y))

In [33]:
def key_press(event):
    global left_up, left_down, right_up, right_down, game_paused, game_over
    
    key = event.keysym.lower()
    
    if key == 'w':
        left_up = True
    elif key == 's':
        left_down = True
    elif key == 'up':
        right_up = True
    elif key == 'down':
        right_down = True
    elif key == 'space':
        game_paused = not game_paused
    elif key == 'r' and game_over:
        restart_game()

In [34]:
def key_release(event):
    global left_up, left_down, right_up, right_down
    
    key = event.keysym.lower()
    
    if key == 'w':
        left_up = False
    elif key == 's':
        left_down = False
    elif key == 'up':
        right_up = False
    elif key == 'down':
        right_down = False

In [35]:
def restart_game():
    global player1_score, player2_score, game_over, game_paused
    player1_score = 0
    player2_score = 0
    game_over = False
    game_paused = False
    reset_ball()

In [36]:
def draw_game():
    canvas.delete("all")
    
    canvas.create_line(WIDTH//2, 0, WIDTH//2, HEIGHT, fill="white", width=2, dash=(5, 5))
    
    canvas.create_rectangle(5, left_pad_y, PAD_W, left_pad_y + PAD_H, fill="white", outline="")
    canvas.create_rectangle(WIDTH - PAD_W, right_pad_y, WIDTH - 5, right_pad_y + PAD_H, fill="white", outline="")
    
    canvas.create_oval(ball_x - BALL_SIZE//2, ball_y - BALL_SIZE//2, 
                      ball_x + BALL_SIZE//2, ball_y + BALL_SIZE//2, fill="white", outline="")
    
    canvas.create_text(WIDTH//4, 50, text=str(player1_score), fill="white", font=("Arial", 36))
    canvas.create_text(3*WIDTH//4, 50, text=str(player2_score), fill="white", font=("Arial", 36))
    
    if game_over:
        winner = "Player 1" if player1_score >= WINNING_SCORE else "Player 2"
        canvas.create_text(WIDTH//2, HEIGHT//2 - 30, text=f"{winner} Wins!", 
                          fill="yellow", font=("Arial", 24))
        canvas.create_text(WIDTH//2, HEIGHT//2 + 10, text="Press 'R' to restart", 
                          fill="white", font=("Arial", 16))
    elif game_paused:
        canvas.create_text(WIDTH//2, HEIGHT//2, text="PAUSED", 
                          fill="yellow", font=("Arial", 24))
        canvas.create_text(WIDTH//2, HEIGHT//2 + 30, text="Press SPACE to continue", 
                          fill="white", font=("Arial", 16))

In [37]:
def game_loop():
    if not game_paused and not game_over:
        update_ball()
        update_paddles()
    
    draw_game()
    root.after(1000 // FPS, game_loop)

In [38]:
root = Tk()
root.title("Pong Game")
root.resizable(False, False)
root.configure(bg="black")

canvas = Canvas(root, width=WIDTH, height=HEIGHT, bg="black", highlightthickness=0)
canvas.pack(padx=10, pady=10)

instructions = Label(root, text="Player 1: W/S keys | Player 2: ↑/↓ keys | SPACE: Pause | R: Restart", 
                    fg="white", bg="black", font=("Arial", 12))
instructions.pack()

root.bind('<KeyPress>', key_press)
root.bind('<KeyRelease>', key_release)
root.focus_set()

In [39]:
reset_ball()
game_loop()
root.mainloop()